# Country Codes & Continents: A Dataset with ISO 3166-1 Alpha-2

This notebook aims to create a dataset of countries, their corresponding ISO 3166-1 Alpha-2 codes, and their respective continents.

**Key Features:**

* Utilizes the ISO 3166-1 Alpha-2 standard for country codes.
* Includes a comprehensive list of countries and their associated continents.
* Provides a clean and organized dataset for various data analysis and mapping projects.

**Potential Use Cases:**

* Geocoding and mapping applications.
* Data analysis and visualization projects.
* Internationalization and localization tasks.
* Building applications that require country-specific information.

This notebook demonstrates a simple and efficient approach to gathering and organizing country-related data.

**Note:** 

* Data sources may vary, and the accuracy of the information should be verified independently.

In [1]:
def fetch_country_data():
    """
    Fetches country data from the REST Countries API and returns a dictionary
    with ISO 3166-1 Alpha-2 codes as keys and details (country name, continent,
    continent code, IT hub status) as values.
    """
    # Mapping of continent full names to continent codes
    continent_codes = {
        "Africa": "AF",
        "Asia": "AS",
        "Europe": "EU",
        "North America": "NA",
        "South America": "SA",
        "Oceania": "OC",
        "Antarctica": "AN"
    }
    
    # List of ISO Alpha-2 codes for IT hub countries
    it_hub_countries = {
        'US', 'IN', 'CN', 'JP', 'KR', 'DE', 'GB', 'FR', 'CA', 'AU',
        'SG', 'SE', 'FI', 'IE', 'IL', 'NL', 'CH', 'ES', 'IT', 'BR',
        'ZA', 'RU', 'AE', 'TR', 'PL', 'VN', 'MY', 'PH', 'TH', 'ID',
        'HK', 'TW'
    }
    
    url = "https://restcountries.com/v3.1/all"
    response = requests.get(url)
    countries = response.json()
    
    country_mapping = {}
    for country in countries:
        country_code = country.get("cca2", None)
        if not country_code:
            continue
        
        country_name = country.get("name", {}).get("common", "Unknown")
        continents = country.get("continents", [])
        if continents:
            continent = continents[0]
            continent_code = continent_codes.get(continent, "Unknown")
        else:
            continent = "Unknown"
            continent_code = "Unknown"
        
        it_hub_status = "Yes" if country_code in it_hub_countries else "No"
        
        country_mapping[country_code] = {
            "country_name": country_name,
            "continent": continent,
            "continent_code": continent_code,
            "it_hub_country": it_hub_status
        }
    
    return country_mapping

In [2]:
def save_country_data(country_data):
    """
    Saves the country data to a JSON file with a timestamped filename.
    """
    timestamp = datetime.datetime.now(ist).strftime("%Y-%m-%d_%H:%M:%S")
    filename = f"RE_{timestamp}_country_details.json"
    filepath = os.path.join(os.getcwd(), filename)  # Save in the current working directory
    with open(filepath, "w", encoding="utf-8") as file:
        json.dump(country_data, file, indent=4, ensure_ascii=False)
    print(f"Country data saved as {filepath}")

In [3]:
    
def PushToGithub(repo_url):
    """
    Automates pushing the generated JSON file to GitHub.
    Finds the latest JSON file, clones/pulls the repository, copies the file,
    commits, and pushes it.
    """
    output_files = os.listdir(os.getcwd())
    # print(output_files)
    try:
        json_files = [file for file in output_files if file.startswith("RE") and file.endswith("country_details.json")]
        if json_files:
            latest_file = max(json_files, key=os.path.getctime)
        else:
            raise ValueError("No JSON files found!")
    except ValueError as e:
        print(f"Error fetching the recent .json file: {e}")
        return False
    
    kaggle_repo_url = os.path.join(os.getcwd(), "YouTubeFoodChannelAnalysis")
    destination_path = os.path.join(kaggle_repo_url, "Requirement", "Daily")
    
    print(f"Latest JSON file: {latest_file}")
    try:
        if os.path.exists(kaggle_repo_url):
            print("Repository already cloned; pulling latest changes.")
            repo = git.Repo(kaggle_repo_url)
            origin = repo.remote(name='origin')
            origin.pull()
            print("Successfully pulled the latest changes.")
        else:
            repo = git.Repo.clone_from(repo_url, kaggle_repo_url)
            print("Successfully cloned the repository.")
        
        if not os.path.exists(destination_path):
            os.makedirs(destination_path)
        shutil.copyfile(os.path.join(os.getcwd(), latest_file),
                        os.path.join(destination_path, latest_file))
        
        repo = Repo(kaggle_repo_url)
        repo.index.add([os.path.join(destination_path, latest_file)])
        timestamp = datetime.datetime.now(ist).strftime("%Y-%m-%d_%H:%M:%S")
        repo.index.commit(f"{timestamp} - Added {latest_file} from Kaggle notebook")
        origin = repo.remote(name="origin")
        push_result = origin.push()
        if push_result:
            print("Output files successfully pushed to GitHub!")
        else:
            print("Pushing to GitHub failed.")
        return True
    except Exception as e:
        print(f"An error occurred during git automation: {e}")
        return False

In [4]:
def main():
    country_data = fetch_country_data()
    save_country_data(country_data)
    PushToGithub(repo_url)

In [5]:
if __name__ == "__main__":
    import requests
    import json
    import datetime
    import os
    from pytz import timezone
    import git
    from git import Repo
    from kaggle_secrets import UserSecretsClient

    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("requirementRepoUrl")
    repo_url = secret_value_0
    ist = timezone('Asia/Kolkata')
    main()

Country data saved as /kaggle/working/RE_2025-02-03_21:16:56_country_details.json
Latest JSON file: RE_2025-02-03_21:16:56_country_details.json
Successfully cloned the repository.
An error occurred during git automation: name 'shutil' is not defined
